In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn  

# Set the tracking URI
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")  # Corrected URI format

# Load the dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Model hyperparameters
params = {
    "penalty": "l2", 
    "solver": "lbfgs", 
    "max_iter": 1000, 
    "multi_class": "auto", 
    "random_state": 8888
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predictions on the test data
y_pred = lr.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Set the tracking URI
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")  # Optional as you've already set it before

# Create a new MLFLOW experiment
mlflow.set_experiment("MLFLOW Quickstart")

# Start an MLFLOW run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)
    
    # Log the accuracy metrics
    mlflow.log_metric("accuracy", accuracy)
    
    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for the iris data")
    
    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))
    
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train[:5],  # Use a subset of training data as an example
        registered_model_name="tracking-quickstart"
    )

    # You can print model info to see its details after logging
    print(model_info)


e:\mlflow startup\my_env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 1.0


Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/12/01 21:15:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2


🏃 View run gentle-dove-112 at: http://127.0.0.1:5000/#/experiments/583353532596579343/runs/eb8e2170ee2c459283ee5ac39e93b910
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/583353532596579343


Created version '2' of model 'tracking-quickstart'.


In [9]:
#infresing and validating the model
from mlflow.models import validate_serving_input

model_uri = 'runs:/8977e997bc154119b1957b6c3ea75903/iris_model'

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      5.9,
      3.2,
      4.8,
      1.8
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([2, 1, 1, 2, 2])

load the model back for prediction as generic python function model


In [10]:
loaded_model = mlflow.pyfunc.load_model(model_uri)

predictions = loaded_model.predict(X_test)
iris_features_name = datasets.load_iris().feature_names
result = pd.DataFrame(X_test,columns=iris_features_name)
result["actual_class"]=y_test
result["predcited_class"]=predictions
result[:5]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predcited_class
0,5.2,3.4,1.4,0.2,0,0
1,4.8,3.0,1.4,0.1,0,0
2,5.8,2.7,5.1,1.9,2,2
3,5.4,3.4,1.7,0.2,0,0
4,7.1,3.0,5.9,2.1,2,2


The MLflow Model Registry is a centralized system for managing the lifecycle of machine learning models. It is part of the MLflow ecosystem, providing tools for versioning, tracking, and organizing models, and making them accessible for collaborative development.

In [11]:
import mlflow.sklearn
# Define model name and version
model_name = "tracking-quickstart"
model_version = "latest"  # Use 'latest' to get the most recent model version

# Construct the model URI
model_uri = f"models:/{model_name}/{model_version}"

# Load the model from the Model Registry
model = mlflow.sklearn.load_model(model_uri)

# Display the model to ensure it's loaded correctly
print(f"Model loaded: {model}")



Model loaded: LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)


e:\mlflow startup\my_env\Lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


In [ ]:
f"models:/{model_name}/{model_version}"


'models:/tracking-quickstart/latest'